In [89]:
import os
import shutil
from glob import glob
import pandas as pd
from tqdm import tqdm

In [3]:
current_path=os.getcwd().replace("\\", "/").replace("c:", "C:")
wav_path=current_path+'/wavFiles/'
trans_path=current_path+"/transcripts/"
dev_path=current_path+"/"+"dev_split_Depression_AVEC2017.csv"
test_path=current_path+"/"+"full_test_split.csv"
train_path=current_path+"/"+"train_split_Depression_AVEC2017.csv"

dev_df=pd.read_csv(dev_path)
test_df=pd.read_csv(test_path)
train_df=pd.read_csv(train_path)

wav_list=glob(wav_path+"*")

test_df=test_df[['Participant_ID', 'PHQ_Binary', 'PHQ_Score', 'Gender']]
test_df.columns=['Participant_ID', 'PHQ8_Binary', 'PHQ8_Score', 'Gender']
train_df=train_df[['Participant_ID', 'PHQ8_Binary', 'PHQ8_Score', 'Gender']]
dev_df=dev_df[['Participant_ID', 'PHQ8_Binary', 'PHQ8_Score', 'Gender']]

In [75]:
id_list=[]
for w in wav_list:
    num=int(w.split("wavFiles\\")[-1].split("_")[0])
    id_list.append(num)
id_list=list(set(id_list))

In [90]:
len(id_list), dev_df.shape, test_df.shape, train_df.shape

(188, (35, 4), (47, 4), (107, 4))

In [81]:
total=pd.concat([dev_df, test_df, train_df])
total["Participant_ID"].nunique(), total.shape
total=total[total["Participant_ID"].isin(id_list)] #아마 다운로드 과정에서 누락된듯 300번
total.reset_index(inplace=True, drop=True)

In [129]:
current_path

'C:/Users/82105/Desktop/코드/english_depression'

In [133]:
# total.to_csv(current_path+"/"+"total.csv", index=False)
# total=pd.read_csv(current_path+"/"+"total.csv")

### 이 모양으로 만들자
- wav_df = pd.DataFrame(
    {'wav_id': wav_list_tmp_id,
     'final_label': wav_list_tmp_label,
     'sentence': wav_list_tmp_sentence
    })

#### transcripts 전처리
- tmp["start_time"]=tmp["start_time"].apply(lambda x:str(x).replace(".","")+".0")
- tmp["stop_time"]=tmp["stop_time"].apply(lambda x:str(x).replace(".","")+".0")

In [113]:
trans_files=glob(trans_path+"*")
new_trans_path=current_path+"/newTransripts/"

In [121]:
for t in tqdm(trans_files):
    filename=t.split("\\")[-1]
    tmp=pd.read_csv(t, sep="\t")
    tmp["start_time"]=tmp["start_time"].apply(lambda x:str(x).replace(".","")+".0")
    tmp["stop_time"]=tmp["stop_time"].apply(lambda x:str(x).replace(".","")+".0")
    tmp.to_csv(new_trans_path+filename, index=False)

100%|██████████| 189/189 [00:01<00:00, 97.51it/s]


#### wav_df 만들기

In [189]:
tmp_list=[]
for w in tqdm(wav_list):
    try:
        fullname=w.split("wavFiles\\")[-1]
        id=fullname.split("_")[0]
        start_time=fullname.split("_")[2]
        start_time=float(start_time.split(".")[0]+".0")
        stop_time=fullname.split("_")[3]
        stop_time=float(stop_time.split(".")[0]+".0")
        tmp=pd.read_csv(new_trans_path+id+"_TRANSCRIPT.csv")
        sentence=tmp[(tmp["start_time"]==start_time)&(tmp["stop_time"]==stop_time)]["value"].values[0]
        
        label=total[total["Participant_ID"]==int(id)]["PHQ8_Binary"].values[0]
        
        tmp_list.append([fullname, label, sentence])
    except:
        pass

100%|██████████| 32718/32718 [00:37<00:00, 878.09it/s] 


In [190]:
wav_df=pd.DataFrame(tmp_list, columns=["filename", "label", "sentence"])

In [192]:
wav_df["label"].value_counts()

label
0    3880
1    1849
Name: count, dtype: int64

In [193]:
wav_df.to_csv(current_path+"wav_df", index=False)